In [ ]:
import os
from collections import defaultdict, OrderedDict
from IPython.display import HTML
import numpy as np
import matplotlib
from matplotlib import animation
import matplotlib.pyplot as plt

from analysis import (
    parse_results, get_token_groups, plot_token_groups, animate, average_length_by_weight,
    tokens, counter_tokens, total_of, sum_up, mk_cumulative_interpolator
)
%matplotlib inline

In [ ]:
token = 'V1'

In [ ]:
def read(path):
    tout, Cout = parse_results(path)
    tokg, ctrg, spg = get_token_groups(Cout)
    init = []
    for sk in spg['polymeric']:
        if Cout[sk][0] > 0:
            init.append((sk, Cout[sk][0]))
    (sk0, ic0), = init
    return dict(tout=tout, Cout=Cout, tokg=tokg, ctrg=ctrg, spg=spg, sk0=sk0, ic0=ic0)

In [ ]:
res = [read('%s-res/%d.txt' % (token, i)) for i in range(10)]

In [ ]:
air = res[1::2]
assert air[0]['Cout']['N2O'][0] == 0
ntr = res[0::2]
assert ntr[0]['Cout']['O2'][0] == 0
series = OrderedDict([('air', air), ('nitrous oxide', ntr)])
for cont in [air, ntr]:
    print([r['ic0'] for r in cont])

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 10))
for ri, (atmo, cont) in enumerate(series.items()):
    axes[ri].set_title(['air', 'nitrous oxide'][ri])
    for r in cont:
        axes[ri].loglog(r['tout'], r['Cout']['H2O2'], label=r['ic0'])
    axes[ri].set_ylim([1e-6, 1e-2])
    axes[ri].legend()

In [ ]:
#plot_token_groups(tout, Cout, tokg)

In [ ]:
#anim1 = animate(tout, Cout, tokg)
#HTML(anim1.to_html5_video())

In [ ]:
sks = 'H2O2 O2 N2O'.split()
fig, axes = plt.subplots(2, 1 + len(sks), figsize=(16, 8))
for ri, (atmo, cont) in enumerate(series.items()):
    for r in cont:
        axes[ri, 0].plot(r['tout'], average_length_by_weight(r['Cout'], r['tokg']), label='%.1g' % r['ic0'])
        for i in range(len(sks)):
            axes[ri, i+1].plot(r['tout'], r['Cout'][sks[i]], label='%.1g' % r['ic0'])
            axes[ri, i+1].set_title(sks[i])
            axes[ri, i+1].set_yscale('log')

            axes[ri, i+1].legend()
        axes[ri, 0].legend()
        axes[ri, 0].set_title(r'Average length by weight')

In [ ]:
fig, axes = plt.subplots(2, len(tokens[1:]), figsize=(16, 8))
for ri, (atmo, cont) in enumerate(series.items()):
    for tk, ax in zip(tokens[1:], axes[ri]):
        for r in cont:
            ax.semilogx(r['tout'], total_of(r['Cout'], r['tokg'][tk])/r['ic0'], label='%.1g' % r['ic0'])
        ax.legend()
        ax.set_title(tk)

In [ ]:
fig, axes = plt.subplots(2, len(counter_tokens), figsize=(16, 8))
for ri, (atmo, cont) in enumerate(series.items()):
    axes[ri, 0].set_ylabel('%s\n\nFraction' % atmo)
    for ct, ax in zip(counter_tokens, axes[ri]):
        for r in cont:
            ax.semilogx(r['tout'], sum_up(r['Cout'], r['ctrg'][ct])/r['ic0'], label='%.1g' % r['ic0'])
        ax.legend()
        ax.set_title(ct)

In [ ]:
import json
dur, varied = json.load(open('varied-%s.json' % token))
dr = varied.pop('doserate')
assert not varied

In [ ]:
cx, cy, ip1d = mk_cumulative_interpolator(dur, dr)

In [ ]:
plt.plot(cx, cy)
plt.xlabel('Time / s')
plt.ylabel('Dose / Gy')

In [ ]:
def plot_vs_doses_and_atmospheres(callback, descr, yscale='linear'):
    doses = [20e3, 40e3, 60e3, 80e3]
    fig, all_axes = plt.subplots(len(doses), 2, figsize=(12, 2*len(doses)), sharey=True)
    for ri, (axes, Dtot) in enumerate(zip(all_axes, doses)):
        ti = np.argmax(cy>=Dtot*(1 - 1e-12))
        axes[0].set_ylabel('D=%.1f kGy\n\n%s' % (cy[ti]/1e3, descr))
        for ci, (ax, (atmo, cont)) in enumerate(zip(axes, series.items())):
            if ri == 0:
                ax.set_title(atmo)
            for r in cont:
                ax.plot(r['ic0']*1e6, callback(r)[ti], marker='o', c='k')
            ax.set_xlabel('Initial concentation of polymer / μM')
    for ax in all_axes.flat:
        ax.set_xlim(left=0)
        ax.set_ylim(bottom=0)
        ax.set_yscale(yscale)
    #fig.tight_layout()

In [ ]:
_avg_size = dict(
    callback=lambda r: average_length_by_weight(r['Cout'], r['tokg']),
    descr='Average length by weight'
)
plot_vs_doses_and_atmospheres(**_avg_size)

In [ ]:
_conc = lambda k: dict(
    callback=lambda r: r['Cout'][k],
    descr='%s / M' % k
)
plot_vs_doses_and_atmospheres(**_conc('H2O2'))

In [ ]:
_principal = lambda k: dict(
    callback=lambda r: total_of(r['Cout'], r['tokg'][k])/r['ic0'],
    descr=k.upper()
)
plot_vs_doses_and_atmospheres(**_principal('a'))

In [ ]:
_counter = lambda k: dict(
    callback=lambda r: sum_up(r['Cout'], r['ctrg'][k])/r['ic0'],
    descr=k
)
plot_vs_doses_and_atmospheres(**_counter('tiny'), yscale='log')

In [ ]:
def dose_plot(callback, descr, *, skip_first=False, axes):
    for idx, (ax, (atmo, cont)) in enumerate(zip(axes, series.items())):
        if skip_first:
            cont = cont[1:]
        ax.set_title(atmo)
        for i, r in enumerate(cont):
            c = np.zeros(3)
            c[idx] = i/(len(cont) - 1)
            ax.plot(ip1d(r['tout'])/1e3, callback(r), c=c, label='%.1g' % r['ic0'])
        ax.legend()
        ax.set_xlabel('Dose / kGy')
        ax.set_ylabel(descr)
        ax.set_xlim(left=0)
        ax.set_ylim(bottom=0)

In [ ]:
%matplotlib inline

In [ ]:
fig, all_axes = plt.subplots(7, 2, figsize=(15, 15)) #, sharey='row')
dose_plot(**_avg_size, axes=all_axes[0, :])
dose_plot(**_principal('l'), axes=all_axes[1, :], skip_first=True)
dose_plot(**_counter('unsaturated'), axes=all_axes[2, :], skip_first=True)
dose_plot(**_conc('H2O2'), axes=all_axes[3, :])
dose_plot(**_conc('O2'), axes=all_axes[4, :])
dose_plot(**_conc('N2O'), axes=all_axes[5, :])
dose_plot(**_counter('damage_fivering'), axes=all_axes[6, :], skip_first=True)
for ax in all_axes[4, :]:
    ax.set_ylim([1e-10, 1e-6])
fig.tight_layout()